In [ ]:
Predicing titanic survived people using machine learning

In [1]:
!pip install pyspark

  Using cached pyspark-3.5.0.tar.gz (316.9 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425388 sha256=d9f7b995084634607b59881308ea288fe72dfdea28fa982d893145f6b2edce37
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\38\df\61\8c121f50c3cffd77f8178180dd232d90b3b99d1bd61fb6d6be
Successfully built pyspark


In [2]:
from pyspark import SparkContext

In [3]:
sc=SparkContext("local", "PySparkIntro")

In [5]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("PySparkIntro").getOrCreate()

In [6]:
data=[1,2,3,4,5]
rdd=sc.parallelize(data)

In [7]:
# Display the content of the original RDD
print("Original RDD:", rdd.collect())

# Modify the RDD by applying a transformation (map)
modified_rdd = rdd.map(lambda x: x + 1)

# Display the content of the modified RDD
print("Modified RDD:", modified_rdd.collect())

# Display the content of the original RDD again
print("Original RDD after modification:", rdd.collect())

Original RDD: [1, 2, 3, 4, 5]
Modified RDD: [2, 3, 4, 5, 6]
Original RDD after modification: [1, 2, 3, 4, 5]


In [8]:
squared_rdd=rdd.map(lambda x:x**2)
squared_rdd

PythonRDD[2] at RDD at PythonRDD.scala:53

In [9]:
even_rdd=rdd.map(lambda x:x%2==0)
even_rdd

PythonRDD[3] at RDD at PythonRDD.scala:53

In [10]:
collected_data=squared_rdd.collect()
collected_data

[1, 4, 9, 16, 25]

In [11]:
num_element=squared_rdd.count()
num_element

5

In [12]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [13]:
schema=StructType([
    StructField("Name",StringType(),True),
    StructField("Age",IntegerType(),True),
    StructField("Salary",IntegerType(),True)
])

In [14]:
data=[("Alice", 28, 45000),("Bob", 36, 60000)]

In [15]:
ds=spark.createDataFrame(data,schema)

In [16]:
ds=ds.alias('Employees')

In [17]:
ds.show()

+-----+---+------+
| Name|Age|Salary|
+-----+---+------+
|Alice| 28| 45000|
|  Bob| 36| 60000|
+-----+---+------+



In [18]:
df=spark.read.csv("E:\\NCI\\Database & Analytics Programming\\Week 4\\train.csv",header=True, inferSchema=True)
df

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string]

In [19]:
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
|          6|       0|     3|    Moran, Mr. James|  male|NULL|    0|    0|      

In [20]:
mean_age=df.select("Age").agg({"Age":"avg"}).collect()[0][0]
mean_fare=df.select("Fare").agg({"Fare":"avg"}).collect()[0][0]

In [21]:
df_cleaned=df.na.fill(mean_age,subset=["Age"])
df_cleaned=df_cleaned.na.fill(mean_fare,subset=["Fare"])

In [22]:
df_cleaned.show()

+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|              Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|             22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|             38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|             26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|             35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|             35.0|    0|    0|          373450|

In [23]:
from pyspark.ml.feature import MinMaxScaler, StandardScaler, VectorAssembler
from pyspark.ml.feature import VectorAssembler

In [26]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer

# Create a Spark session
spark = SparkSession.builder.appName("LabelEncoderExample").getOrCreate()

# Assuming 'cleaned_df' is your PySpark DataFrame
# Create a copy to avoid modifying the original DataFrame
encoded_df = df_cleaned

# Initialize StringIndexer with input and output column names
string_indexer = StringIndexer(inputCol="Sex", outputCol="sex_encoded")

# Fit and transform the DataFrame
encoded_df = string_indexer.fit(encoded_df).transform(encoded_df)

# Display the updated DataFrame
encoded_df.show()

+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+-----------+
|PassengerId|Survived|Pclass|                Name|   Sex|              Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|sex_encoded|
+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+-----------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|             22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|        0.0|
|          2|       1|     1|Cumings, Mrs. Joh...|female|             38.0|    1|    0|        PC 17599|71.2833|  C85|       C|        1.0|
|          3|       1|     3|Heikkinen, Miss. ...|female|             26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|        1.0|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|             35.0|    1|    0|          113803|   53.1| C123|       S|        1.0|
|          5|       

In [29]:
assembler=VectorAssembler(inputCols=['Age','Fare','sex_encoded'],outputCol='Features')
assembled_df = assembler.transform(encoded_df)
assembled_df.show()

+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+-----------+--------------------+
|PassengerId|Survived|Pclass|                Name|   Sex|              Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|sex_encoded|            Features|
+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+-----------+--------------------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|             22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|        0.0|     [22.0,7.25,0.0]|
|          2|       1|     1|Cumings, Mrs. Joh...|female|             38.0|    1|    0|        PC 17599|71.2833|  C85|       C|        1.0|  [38.0,71.2833,1.0]|
|          3|       1|     3|Heikkinen, Miss. ...|female|             26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|        1.0|    [26.0,7.925,1.0]|
|          4|       1|     1|Futre

In [30]:
min_max_scaler = MinMaxScaler(inputCol="Features", outputCol="minmax_scaled_features")
min_max_model = min_max_scaler.fit(assembled_df)
min_max_scaled_df = min_max_model.transform(assembled_df)

In [31]:
min_max_scaled_df.show()

+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+-----------+--------------------+----------------------+
|PassengerId|Survived|Pclass|                Name|   Sex|              Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|sex_encoded|            Features|minmax_scaled_features|
+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+-----+--------+-----------+--------------------+----------------------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|             22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|        0.0|     [22.0,7.25,0.0]|  [0.27117366172405...|
|          2|       1|     1|Cumings, Mrs. Joh...|female|             38.0|    1|    0|        PC 17599|71.2833|  C85|       C|        1.0|  [38.0,71.2833,1.0]|  [0.47222920331741...|
|          3|       1|     3|Heikkinen, Miss. ...|female|             26.0|    0

In [32]:
# StandardScaler
standard_scaler = StandardScaler(inputCol="Features", outputCol="standard_scaled_features", withStd=True, withMean=True)
standard_model = standard_scaler.fit(assembled_df)
standard_scaled_df = standard_model.transform(assembled_df)

In [33]:
train_data, test_data = standard_scaled_df.randomSplit([0.7, 0.3])

In [34]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.sql.functions import col
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.util import MLUtils

rf = DecisionTreeClassifier(featuresCol='standard_scaled_features', labelCol='Survived')
# Training the model
model = rf.fit(train_data)
# Making predictions on test data
predictions = model.transform(test_data)

# Evaluation
evaluator = MulticlassClassificationEvaluator(labelCol='Survived', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)

precision_recall_f1 = MulticlassClassificationEvaluator(labelCol='Survived', metricName='weightedPrecision')\
                        .evaluate(predictions),\
                    MulticlassClassificationEvaluator(labelCol='Survived', metricName='weightedRecall')\
                        .evaluate(predictions),\
                    MulticlassClassificationEvaluator(labelCol='Survived', metricName='f1')\
                        .evaluate(predictions)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision_recall_f1[0]}")
print(f"Recall: {precision_recall_f1[1]}")
print(f"F1-score: {precision_recall_f1[2]}")

Accuracy: 0.805668016194332
Precision: 0.8043298129682487
Recall: 0.805668016194332
F1-score: 0.8047782177336833
